# Load Dependencies 

In [71]:
%%capture
!pip install kaggle
import tensorflow as tf
import keras
import numpy as np
import tensorflow_hub
import tensorflow_datasets as tfds
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import copy
import os
!pip install timm
import timm

In [4]:
!cp -f ./kaggle.json /root/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json

In [5]:
%%capture
!kaggle datasets download -d moltean/fruits
!unzip fruits.zip

# Pytorch Equivalent of the Model(MobileNetv2 Transfer Learning)

In [33]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch.optim as optim
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')

In [41]:
transforms = torchvision.transforms.Compose([
  torchvision.transforms.ToTensor(),
  #torchvision.transforms.ColorJitter(brightness = 0.1, contrast = 0.1, saturation = 0.1, hue = 0.1),
  torchvision.transforms.RandomHorizontalFlip(),
  torchvision.transforms.RandomVerticalFlip(),
  torchvision.transforms.Resize((224, 224)),
  torchvision.transforms.RandomRotation(30),
  torchvision.transforms.RandomGrayscale()
])

In [91]:
imagefolder = torchvision.datasets.ImageFolder("./fruits-360/Training", transform = transforms)

In [92]:
train, val = torch.utils.data.random_split(imagefolder, [len(imagefolder) - 32, 32])

In [101]:
trainloader = torch.utils.data.DataLoader(train, batch_size = 32, shuffle = True)
valloader = torch.utils.data.DataLoader(val, batch_size = 32)

In [130]:
class MobileNetV2(nn.Module):
  def __init__(self):
    super().__init__()
    self.model = timm.create_model('efficientnet_b0', pretrained = True)
    self.model.fc = nn.Linear(2048, 131)
  def forward(self, x):
    vals= self.model(x)
    return vals

In [131]:
class Solver(nn.Module):
  def __init__(self):
    super().__init__()
    self.model = MobileNetV2()
    self.optim = optim.Adam(self.model.parameters(), lr = 1e-3)
    self.criterion = nn.CrossEntropyLoss()
  def training_loop(self, trainloader, valloader, num_epochs, display_every = 16):
    for val_images, val_labels in valloader:
      val_images = val_images.to(device)
      val_labels = val_labels.to(device)
      break
    for epoch in range(num_epochs):
      count = 0
      total_loss = 0
      self.train()
      for images, labels in trainloader:
        images = images.to(device)
        labels = labels.to(device)
        self.optim.zero_grad()
        pred = self.model(images)
        _, max = torch.max(pred, dim = -1)
        #print(max)
        #print(labels)
        loss = self.criterion(pred, labels)
        loss.backward()
        total_loss += loss.item()
        count += 1
        self.optim.step()
        del images
        del labels
        del pred
        del loss
        torch.cuda.empty_cache()
        if count == display_every:
          break
      print(f"EPOCH {epoch}, total_loss: {total_loss / count}")
      self.eval()
      with torch.no_grad():
        pred = F.softmax(self.model(val_images), dim = -1)
        _, max = torch.max(pred, dim = -1)
        #print(max)
        #print(val_labels)
        print(torch.sum((max == val_labels).int()) / val_labels.shape[0])

In [132]:
solver = Solver()
solver.to(device)
solver.training_loop(trainloader, valloader, 100)

EPOCH 0, total_loss: 5.418253138661385
tensor(0.1250, device='cuda:0')
EPOCH 1, total_loss: 2.606282502412796
tensor(0.5625, device='cuda:0')
EPOCH 2, total_loss: 1.4726136326789856
tensor(0.6875, device='cuda:0')
EPOCH 3, total_loss: 0.9307772945612669
tensor(0.7812, device='cuda:0')
EPOCH 4, total_loss: 0.8352786768227816
tensor(0.8125, device='cuda:0')
EPOCH 5, total_loss: 0.6810991503298283
tensor(0.7500, device='cuda:0')
EPOCH 6, total_loss: 0.5335297239944339
tensor(0.8750, device='cuda:0')
EPOCH 7, total_loss: 0.48340775817632675
tensor(0.8750, device='cuda:0')
EPOCH 8, total_loss: 0.3296648608520627
tensor(0.9062, device='cuda:0')
EPOCH 9, total_loss: 0.3355043902993202
tensor(0.9688, device='cuda:0')
EPOCH 10, total_loss: 0.30898548755794764
tensor(0.9688, device='cuda:0')
EPOCH 11, total_loss: 0.3221653215587139
tensor(0.8750, device='cuda:0')
EPOCH 12, total_loss: 0.24055460467934608
tensor(0.9688, device='cuda:0')
EPOCH 13, total_loss: 0.2850480149500072
tensor(0.9375, devi

KeyboardInterrupt: ignored

# Load Dataset In.
I will build a custom dataset object and will apply various data augmentation strategies.


In [133]:
class Fruits360Dataset(keras.utils.Sequence):
  def __init__(self, x_set, y_set):
    self.x = x_set
    self.y = y_set
  def __len__(self):
    return len(self.x)
  def __getitem__(self, index):
    batch_x = self.x[index]
    batch_y = self.y[index]
    return batch_x, batch_y

In [167]:
dataset = keras.preprocessing.image_dataset_from_directory("./fruits-360/Training", batch_size = 32)
# Load images into train and validation
train_dataset = dataset.take(2115)
val_dataset = dataset.skip(2115)
test_dataset = keras.preprocessing.image_dataset_from_directory("./fruits-360/Test")
train_dataset = train_dataset.map(lambda x, y:(tf.image.resize(x, (224, 224)), y))
val_dataset = val_dataset.map(lambda x, y: (tf.image.resize(x, (224, 224)), y))
test_dataset = test_dataset.map(lambda x, y: (tf.image.resize(x, (224, 224)), y))

Found 67692 files belonging to 131 classes.
Found 22688 files belonging to 131 classes.


In [168]:
num_classes = len(os.listdir("./fruits-360/Training"))

Data Augmentation

In [169]:
transforms = keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomRotation(30, fill_mode = 'constant'),
  tf.keras.layers.experimental.preprocessing.RandomFlip()
])
def augment(x):
  x = transforms(x)
  x = tf.repeat(tf.reduce_sum(x, axis = 3) / 3.0, 3, axis = -1) if random.random() < 0.1 else x
  return tf.image.resize(x, (224, 224))
# Map TrainSet to other tranforms
train_dataset = train_dataset.map(lambda x, y: (augment(x), y))

In [170]:
def display_images(dataloader):
  for images, _ in dataloader.take(1):
    for image in images:
      plt.imshow(image/ 255)
      plt.show()

In [171]:
#display_images(train_dataset)

In [172]:
class BatchNormBlock(tf.keras.layers.Layer):
  def __init__(self, out_channels, kernel_size, stride):
    super().__init__()
    self.out_channels = out_channels
    self.conv = keras.layers.Conv2D(out_channels, kernel_size, strides = stride, padding = 'same')
    self.bn = keras.layers.BatchNormalization()
  def call(self, x):
    return self.bn(keras.activations.relu(self.conv(x)))

Build Model 

In [173]:
model = tf.keras.applications.EfficientNetB0(include_top=False, input_shape = (224, 224, 3))
for i in model.layers[0:80]:
  i.trainable = False
sequential = keras.Sequential([
    model,
    keras.layers.AvgPool2D(pool_size = 7),
    keras.layers.Flatten(),
    keras.layers.Dense(131, activation = 'softmax')
])

In [174]:
sequential.compile(optimizer = keras.optimizers.Adam(learning_rate= keras.optimizers.schedules.ExponentialDecay(1e-3, 5, 0.9)), loss = keras.losses.SparseCategoricalCrossentropy(), metrics = ['accuracy'])

In [175]:
for images, labels in train_dataset:
  print(images.shape)
  break

(32, 224, 224, 3)


In [176]:
for images, labels in train_dataset:
  features = sequential(images)
  print(features.shape)
  break

(32, 131)


In [177]:
sequential.fit(train_dataset, epochs = 200, batch_size = 32, steps_per_epoch=64)

Epoch 1/200
64/64 [==============================] - 26s 288ms/step - loss: 3.6340 - accuracy: 0.2985
Epoch 2/200
64/64 [==============================] - 18s 289ms/step - loss: 0.8504 - accuracy: 0.8402
Epoch 3/200
64/64 [==============================] - 19s 290ms/step - loss: 0.4946 - accuracy: 0.9291
Epoch 4/200
64/64 [==============================] - 18s 287ms/step - loss: 0.4420 - accuracy: 0.9330
Epoch 5/200
64/64 [==============================] - 18s 288ms/step - loss: 0.4464 - accuracy: 0.9235
Epoch 6/200
64/64 [==============================] - 18s 289ms/step - loss: 0.4337 - accuracy: 0.9335
Epoch 7/200
64/64 [==============================] - 18s 287ms/step - loss: 0.4236 - accuracy: 0.9433
Epoch 8/200
64/64 [==============================] - 18s 286ms/step - loss: 0.3946 - accuracy: 0.9584
Epoch 9/200
64/64 [==============================] - 18s 285ms/step - loss: 0.3939 - accuracy: 0.9421
Epoch 10/200
64/64 [==============================] - 18s 288ms/step - loss: 0.395

KeyboardInterrupt: ignored

Evaluate the Model on the test Set

In [178]:
scores = sequential.evaluate(test_dataset, batch_size = 32)

709/709 [==============================] - 64s 88ms/step - loss: 0.3972 - accuracy: 0.9325


In [181]:
sequential.save_weights("./model/model")